In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import shutil
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix
import random
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,BatchNormalization,Flatten,AvgPool2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
#!pip install keras-tuner -q


In [3]:
#import keras_tuner as kt


In [4]:
path='C:/Users/user/Desktop/Cours MPDS2 S1/DATASET'
target=os.listdir(os.path.join(path,'TRAIN'))
print(target)

['O', 'R']


In [5]:
new_dir='C:/Users/user/Desktop/Cours MPDS2 S1/data'
train_path=new_dir+'/train'
valid_path=new_dir+'/valid'
test_path=new_dir+'/test'

In [6]:
def create_dir():
    if not os.path.isdir(new_dir):
        os.makedirs(new_dir)
        os.makedirs(train_path)
        os.makedirs(valid_path)
        os.makedirs(test_path)
        for label in target:
            os.makedirs(os.path.join(train_path,label))
            os.makedirs(os.path.join(valid_path,label))
            os.makedirs(os.path.join(test_path,label))

In [7]:
create_dir()

In [8]:
def check_path():
    print(f'{new_dir}: {os.path.isdir(new_dir)}')
    print(f'{train_path}:{os.path.isdir(train_path)}')
    print(f'{valid_path}:{os.path.isdir(valid_path)}')
    print(f'{test_path}:{os.path.isdir(test_path)}')
    for label in target:
        print(f'{os.path.join(train_path,label)}:{os.path.isdir(os.path.join(train_path,label))}')
        print(f'{os.path.join(valid_path,label)}:{os.path.isdir(os.path.join(valid_path,label))}')
        print(f'{os.path.join(test_path,label)}:{os.path.isdir(os.path.join(test_path,label))}')


In [9]:
check_path()

C:/Users/user/Desktop/Cours MPDS2 S1/data: True
C:/Users/user/Desktop/Cours MPDS2 S1/data/train:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/valid:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/test:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/train\O:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/valid\O:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/test\O:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/train\R:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/valid\R:True
C:/Users/user/Desktop/Cours MPDS2 S1/data/test\R:True


In [10]:
def find_image_size():
    image_path=os.path.join(path,'TRAIN/O')
    for image in os.listdir(image_path):
        img_path=os.path.join(image_path,image)
        img=cv2.imread(img_path)
        print(img.shape)
        break

In [11]:
find_image_size()

(242, 208, 3)


In [12]:
img_size=(96,96) 
batch_size=20
epochs=30
train_valid_path=path+'/TRAIN' 
testing_path=path+'/TEST'
steps_per_epoch=250

In [13]:
def load_train_images(img_per_category=3500):
    for folder in os.listdir(train_valid_path):
        folder_path=os.path.join(train_valid_path,folder)
        print(f'Loading training images for {folder}')
        train_image_set=random.sample(os.listdir(folder_path),img_per_category)
        dest=os.path.join(train_path,folder)
        for c in tqdm(train_image_set):
            img_path=os.path.join(folder_path,c)
            shutil.copy(img_path,dest)
def load_valid_images(img_per_category=1000):
    for folder in os.listdir(train_valid_path):
        folder_path=os.path.join(train_valid_path,folder)
        print(f'Loading validation images for {folder}')
        dest=os.path.join(valid_path,folder)
        valid_image_set=random.sample(os.listdir(folder_path),img_per_category)
        for c in tqdm(valid_image_set):
            img_path=os.path.join(folder_path,c)
            shutil.copy(img_path,dest)
def load_test_images(img_per_category=500):
    for folder in os.listdir(testing_path):
        print(f'Loading the testing images for {folder}')
        folder_path=os.path.join(testing_path,folder)
        dest=os.path.join(test_path,folder)
        test_image_set=random.sample(os.listdir(folder_path),img_per_category)
        for c in tqdm(test_image_set):
            img_path=os.path.join(folder_path,c)
            shutil.copy(img_path,dest)

In [14]:
load_train_images(5000)

  0%|          | 5/5000 [00:00<02:14, 37.21it/s]

Loading training images for O


  0%|          | 12/5000 [00:00<00:46, 107.26it/s]

Loading training images for R


100%|██████████| 5000/5000 [01:23<00:00, 59.98it/s] 


In [15]:
load_valid_images(1500)



  0%|          | 0/1500 [00:00<?, ?it/s]

Loading validation images for O


  2%|▏         | 26/1500 [00:00<00:07, 202.16it/s]

Loading validation images for R


100%|██████████| 1500/1500 [00:13<00:00, 111.04it/s]


In [16]:
load_test_images(600)

  0%|          | 2/600 [00:00<00:48, 12.36it/s]

Loading the testing images for O


  1%|          | 5/600 [00:00<00:14, 40.00it/s]

Loading the testing images for R


100%|██████████| 600/600 [00:10<00:00, 58.81it/s]


In [17]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rescale=1./255)
train_batch=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rescale=1./255,horizontal_flip=True).flow_from_directory(directory=train_path,target_size=img_size,classes=target,batch_size=batch_size)
valid_batch=datagen.flow_from_directory(directory=valid_path,target_size=img_size,classes=target,batch_size=batch_size)
test_batch=datagen.flow_from_directory(directory=test_path,target_size=img_size,classes=target,batch_size=batch_size,shuffle=False)

Found 15512 images belonging to 2 classes.
Found 5593 images belonging to 2 classes.
Found 1535 images belonging to 2 classes.


In [18]:
images,labels=train_batch.next()
print(np.max(images))

0.59239614


In [19]:
def myModel(img):
    layer=int(np.log2(img))
    model=[]
    model+=[
        Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='same',input_shape=(img,img,3),activation='relu'),
        BatchNormalization(),
        Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='same',activation='relu'),
        BatchNormalization(),
        AvgPool2D(pool_size=(2,2),strides=2),
    ]
    layer-=1
    filters=64
    for i in range(layer):
        model+=[
            Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding='same',activation='relu'),
            BatchNormalization(),
            Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding='same',activation='relu'),
            BatchNormalization(),
            AvgPool2D(pool_size=(2,2),strides=2),
        ]
        filters*=2
    model+=[
        Flatten(),
        Dense(units=4096,activation='relu'),
        Dense(units=4096,activation='relu'),
        Dense(units=len(target),activation='softmax')
    ]
    return Sequential(model)

In [20]:
def build_model(hp):
    model=myModel(96)
    hp_lr=hp.Choice('learning_rate',values=list(np.linspace(start=0.0001,stop=0.0006,num=12)))
    model.compile(optimizer=Adam(lr=hp_lr),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [21]:
tuner=kt.Hyperband(build_model,objective='val_accuracy',max_epochs=epochs)


INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [22]:
stop_early=EarlyStopping(monitor='val_loss',patience=5)


In [ ]:
tuner.search(train_batch,epochs=3,validation_data=valid_batch,callbacks=[stop_early])



Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.00028182        |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |3                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2
 14/776 [..............................] - ETA: 1:05:24 - loss: 1.3411 - accuracy: 0.6464